In [1]:
import os
import qiime2
import numpy as np

from qiime2.plugins.feature_table.methods import group

In [2]:
base_dir = "/Users/gregcaporaso/Dropbox/shared-analysis-results/2019.09-exmp/combined/"
sample_md_fp = os.path.join(base_dir, "sample-metadata", "sample-metadata.tsv")
data_dir = os.path.join(base_dir, "dada2-paired-long/")
table_fp = os.path.join(data_dir, "table-sepp-no-excludes.qza")

In [3]:
sample_md = qiime2.Metadata.load(sample_md_fp).to_dataframe()
table = qiime2.Artifact.load(table_fp)

In [4]:
def week_to_period(week):
    week = float(week)
    if np.isnan(week): 
        return week
    elif week < 4:
        return '1'
    elif week < 7:
        return '2'
    elif week < 9: 
        return '3'
    elif week < 12:
        return '4'
    else:
        return '5'

In [5]:
sample_md['period'] = list(map(week_to_period, sample_md['week']))
grouped_sample_metadata = sample_md[['period', 'subject-id', 'project', 'activity', 'exclude']]
grouped_sample_metadata['subject-id-period'] = ['-'.join(map(str,e)) for e in zip(grouped_sample_metadata['subject-id'], grouped_sample_metadata['period'])]

/Users/gregcaporaso/miniconda3/envs/qiime2-2019.10/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
grouped_table = group(table, 
                      axis='sample', 
                      metadata=qiime2.CategoricalMetadataColumn(grouped_sample_metadata['subject-id-period']), 
                      mode='sum').grouped_table

In [7]:
grouped_sample_metadata = grouped_sample_metadata.drop_duplicates(subset='subject-id-period')
grouped_sample_metadata = grouped_sample_metadata.set_index('subject-id-period')
grouped_sample_metadata.index.name = 'sample-id'

In [8]:
output_table_fp = os.path.join(data_dir, "table-sepp-no-excludes-grouped-by-period.qza")
grouped_table.save(output_table_fp) 

output_metadata_fp = os.path.join(data_dir, "sample-metadata-grouped-by-period.tsv")
qiime2.Metadata(grouped_sample_metadata).save(output_metadata_fp)